<a href="https://colab.research.google.com/github/Fu-kit/resubmission-assessment2-weather-CSC101/blob/main/Resubmission_assessment2_weather_CSC101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# =====================================================
# Cell 1: Install dependencies
# =====================================================

!pip -q install fetch-my-weather spacy pyinputplus
!python -m spacy download en_core_web_sm -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [38]:
# =====================================================
# Cell 2: Imports and load spaCy
# =====================================================

import pyinputplus as pyip
import spacy
from fetch_my_weather import get_weather

# Load spaCy model
nlp = spacy.load("en_core_web_sm")


In [39]:
# =====================================================
# Cell 3: Helpers (detect day, rain signals)
# =====================================================
import datetime as dt

def _when_from_text(q: str) -> str:
    q = (q or "").lower()
    if "tomorrow" in q: return "tomorrow"
    if "weekend" in q:  return "weekend"
    return "today"

def _day_index(weather_days, when_word: str) -> int:
    if when_word == "tomorrow": return 1 if len(weather_days) > 1 else 0
    if when_word == "weekend":
        for i, d in enumerate(weather_days[:3]):
            try:
                wd = dt.date.fromisoformat(d["date"]).weekday()
                if wd >= 5: return i
            except: pass
        return 0
    return 0

def _best_rain_signal(hourly):
    max_ch, max_pp = 0, 0.0
    for h in (hourly or []):
        try: ch = int(h.get("chanceofrain", h.get("chanceofprecip", "0")) or 0)
        except: ch = 0
        try: pp = float(h.get("precipMM", 0) or 0.0)
        except: pp = 0.0
        if (ch, pp) > (max_ch, max_pp):
            max_ch, max_pp = ch, pp
    return max_ch, max_pp


In [40]:
# =====================================================
# Cell 4: Core weather functions
# =====================================================

def get_weather_data(location, days=2):
    return get_weather(location=location, format="raw_json")

def parse_weather_question(question):
    text = (question or "").lower()
    doc = nlp(text)

    intent = None
    if any(w in text for w in ["umbrella", "rain", "wet", "showers"]):
        intent = "umbrella"
    elif any(w in text for w in ["cold", "temperature", "hot", "warm"]):
        intent = "temperature"
    elif "picnic" in text:
        intent = "picnic"
    elif any(w in text for w in ["wear", "clothes", "jacket", "coat"]):
        intent = "clothing"

    when = _when_from_text(text)

    import re
    m = re.search(r"\b(?:in|for|at)\s+([a-zA-Z][\w\s\.-]*(?:,\s*[A-Z]{2,3})?)\b", text)
    loc_in_text = m.group(1).strip() if m else None

    return {"intent": intent, "when": when, "location_in_text": loc_in_text}

def generate_weather_response(parsed, weather_data):
    when = parsed.get("when", "today")
    days = weather_data.get("weather", [])
    if not days:
        return "Sorry, I couldn't read the forecast."

    idx = _day_index(days, when)
    day = days[min(idx, len(days)-1)]
    hourly = day.get("hourly", [])
    max_ch, max_pp = _best_rain_signal(hourly)

    avgtemp = day.get("avgtempC") or "N/A"
    cond = "N/A"
    if hourly:
        cond = hourly[len(hourly)//2].get("weatherDesc", [{"value":"N/A"}])[0]["value"]
    else:
        try:
            cond = weather_data["current_condition"][0]["weatherDesc"][0]["value"]
        except:
            pass

    intent = parsed.get("intent")
    if intent == "umbrella":
        need = (max_ch >= 40) or (max_pp >= 0.2)
        ans = "Yes, take an umbrella." if need else "No, you probably won't need an umbrella."
        return f"{ans} ({when}). Rain ~{max_pp} mm | {max_ch}%. {cond}, {avgtemp}°C."
    if intent == "temperature":
        return f"{when.capitalize()}: {avgtemp}°C. {cond}. Rain ~{max_pp} mm | {max_ch}%."
    if intent == "picnic":
        ok = (max_ch < 30) and (float(avgtemp if str(avgtemp).isdigit() else 20) >= 18)
        return f"{'Good' if ok else 'Not ideal'} for picnic {when}. {cond}, {avgtemp}°C; rain {max_ch}%."
    if intent == "clothing":
        tips = []
        try: t = float(avgtemp)
        except: t = 20.0
        if t >= 28: tips.append("short sleeves")
        elif t >= 22: tips.append("light layer")
        else: tips.append("warm layer")
        if max_ch >= 40: tips.append("rain jacket")
        return f"{when.capitalize()} wear: {', '.join(tips)}. {cond}, {avgtemp}°C; rain {max_ch}%."

    return f"{when.capitalize()}: {cond}, {avgtemp}°C; rain ~{max_pp} mm | {max_ch}%."


In [ ]:
# =====================================================
# Cell 5: Console (location-first) + ASCII weather icons
#   - uses get_weather_data / parse_weather_question / generate_weather_response
#   - NO direct WeatherWrapper usage
# =====================================================

# Small ASCII icon set (wttr.in-style)
ASCII_ICONS = {
    "clear": r"""
      \   /
       .-.
    ― (   ) ―
       `-’
      /   \
""",
    "cloudy": r"""
             .--.
          .-(    ).
         (___.__)__)
""",
    "rain": r"""
     .-.
    (   ).
   (___(__)
    ʻ ʻ ʻ ʻ
    ʻ ʻ ʻ ʻ
""",
    "snow": r"""
     .-.
    (   ).
   (___(__)
    * * * *
   * * * *
""",
    "thunder": r"""
     .-.
    (   ).
   (___(__)
    ‚⚡‚⚡‚⚡‚
    ‚⚡‚⚡‚⚡‚
"""
}

def _icon_for_condition(cond: str) -> str:
    ck = (cond or "").lower()
    if "thunder" in ck or "storm" in ck or "lightning" in ck:
        return ASCII_ICONS["thunder"]
    if "snow" in ck or "sleet" in ck or "blizzard" in ck:
        return ASCII_ICONS["snow"]
    if "rain" in ck or "shower" in ck or "drizzle" in ck:
        return ASCII_ICONS["rain"]
    if "cloud" in ck or "overcast" in ck:
        return ASCII_ICONS["cloudy"]
    return ASCII_ICONS["clear"]

def _condition_for_when(data: dict, when_word: str) -> str:
    """Pick a representative condition for the requested day."""
    days = data.get("weather", [])
    if not days:
        # fallback to current condition if available
        try:
            return data["current_condition"][0]["weatherDesc"][0]["value"]
        except Exception:
            return "Clear"
    # Use helper from Cell 3 to choose index
    idx = _day_index(days, when_word)
    day = days[min(idx, len(days)-1)]
    hourly = day.get("hourly", [])
    if hourly:
        # take a mid-day-ish slot if present
        mid = hourly[len(hourly)//2]
        try:
            return mid.get("weatherDesc", [{"value":"N/A"}])[0]["value"]
        except Exception:
            pass
    # fallback again
    try:
        return data["current_condition"][0]["weatherDesc"][0]["value"]
    except Exception:
        return "Clear"

def run_location_first_console():
    print("\nWeatherWise — location first + ASCII icons\n")
    # 1) Location
    while True:
        location = input("Enter a location (City or Country): ").strip()
        if location:
            break
        print("Please enter a location.\n")

    # Initial fetch (uses get_weather_data from Cell 4)
    try:
        data = get_weather_data(location, days=3)
    except Exception as e:
        print(f"Error fetching weather: {e}")
        return

    # 2) Q&A loop
    while True:
        q = input("\nAsk your question (e.g., 'Should I take an umbrella tomorrow?')\n(Press Enter to exit)\n> ").strip()
        if not q:
            print("Goodbye!")
            break

        parsed = parse_weather_question(q)  # from Cell 4

        # If the user wrote "in <city>" in the question, switch data to that city
        use_loc = parsed.get("location_in_text") or location
        if use_loc != location:
            try:
                data = get_weather_data(use_loc, days=3)
                location = use_loc
            except Exception as e:
                print(f"Error fetching weather for '{use_loc}': {e}")
                continue

        # Compose and print the textual response
        resp = generate_weather_response(parsed, data)  # from Cell 4
        print("\n" + resp)

        # Show ASCII icon for the asked day
        cond = _condition_for_when(data, parsed.get("when", "today"))
        print(_icon_for_condition(cond))

# Run it
run_location_first_console()



WeatherWise — location first + ASCII icons

Enter a location (City or Country): Perth

Ask your question (e.g., 'Should I take an umbrella tomorrow?')
(Press Enter to exit)
> Do I need to bring an umbrella?

No, you probably won't need an umbrella. (today). Rain ~0.0 mm | 0%. Partly Cloudy , 14°C.

             .--.    
          .-(    ).  
         (___.__)__) 

